In [1]:
import duckdb
import pandas as pd
import os

In [2]:
df_epci = duckdb.read_csv("./data/raw/perimetre-epci-a-fp.csv", sep=";",ignore_errors=True)
df_epci

┌─────────┬───────────┬────────────────────────────────────┬──────────────────┬─────────────┬────────────┬───────────────┬───────────────┬─────────┬─────────┬──────────────┬────────────────────────┬───────────┬───────────┐
│  dept   │   siren   │           raison_sociale           │ nature_juridique │ mode_financ │ nb_membres │ total_pop_tot │ total_pop_mun │ dep_com │  insee  │ siren_membre │       nom_membre       │ ptot_2025 │ pmun_2025 │
│ varchar │   int64   │              varchar               │     varchar      │   varchar   │   int64    │    varchar    │    varchar    │ varchar │ varchar │    int64     │        varchar         │  varchar  │  varchar  │
├─────────┼───────────┼────────────────────────────────────┼──────────────────┼─────────────┼────────────┼───────────────┼───────────────┼─────────┼─────────┼──────────────┼────────────────────────┼───────────┼───────────┤
│ 01      │ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ CC               │ FPU         │         14 │ 1

In [10]:
query_epci = """
SELECT 
    DISTINCT siren,
    dept,
    TRY_CAST(REPLACE(total_pop_tot,' ','') AS DOUBLE) AS total_pop_tot,
    nb_membres
FROM df_epci
WHERE total_pop_tot IS NOT NULL
ORDER BY siren
"""

df_epci_filtered = duckdb.sql(query_epci)
df_epci_filtered

┌───────────┬─────────┬───────────────┬────────────┐
│   siren   │  dept   │ total_pop_tot │ nb_membres │
│   int64   │ varchar │    double     │   int64    │
├───────────┼─────────┼───────────────┼────────────┤
│ 200000172 │ 74      │       29204.0 │          7 │
│ 200000438 │ 44      │       37415.0 │          9 │
│ 200000545 │ 10      │       19206.0 │          6 │
│ 200000628 │ 84      │       24753.0 │          5 │
│ 200000800 │ 41      │       10410.0 │          6 │
│ 200000925 │ 21      │       22568.0 │         22 │
│ 200000933 │ 18      │       14561.0 │         14 │
│ 200004802 │ 83      │       30281.0 │          9 │
│ 200005932 │ 45      │       15994.0 │          7 │
│ 200005957 │ 88      │       13184.0 │         30 │
│     ·     │ ·       │          ·    │          · │
│     ·     │ ·       │          ·    │          · │
│     ·     │ ·       │          ·    │          · │
│ 249720053 │ 972     │      116446.0 │         12 │
│ 249720061 │ 972     │      151772.0 │       

In [4]:
df_com = duckdb.read_csv("./data/processed/communes-france-2025-processed.csv",sep=",")
df_com

┌────────────┬─────────────────────────┬─────────────────────────┬──────────┬──────────────────────┬──────────┬──────────────┬───────────┬───────────────────────────────────────────┬────────────┬────────────────┬─────────────────┐
│ code_insee │      nom_standard       │     nom_sans_accent     │ reg_code │       reg_nom        │ dep_code │   dep_nom    │ epci_code │                 epci_nom                  │ population │ superficie_km2 │ densite_hab_km2 │
│  varchar   │         varchar         │         varchar         │  int64   │       varchar        │ varchar  │   varchar    │  varchar  │                  varchar                  │   int64    │     int64      │     double      │
├────────────┼─────────────────────────┼─────────────────────────┼──────────┼──────────────────────┼──────────┼──────────────┼───────────┼───────────────────────────────────────────┼────────────┼────────────────┼─────────────────┤
│ 01001      │ L'Abergement-Clémenciat │ l-abergement-clemenciat │       84 

In [16]:
query = """
SELECT 
    epci_code,
    sum(superficie_km2) AS superficie_km2
FROM df_com
WHERE (superficie_km2 IS NOT NULL) AND (epci_code != 'ZZZZZZZZZ')
GROUP BY epci_code
"""

df_surface_epci = duckdb.sql(query)
df_surface_epci

┌───────────┬────────────────┐
│ epci_code │ superficie_km2 │
│  varchar  │     int128     │
├───────────┼────────────────┤
│ 200040350 │            437 │
│ 200071751 │           1208 │
│ 200070308 │            300 │
│ 240200501 │            369 │
│ 200040426 │            330 │
│ 200070985 │            265 │
│ 240300491 │            333 │
│ 240300657 │            380 │
│ 200071512 │            733 │
│ 200071884 │            917 │
│     ·     │             ·  │
│     ·     │             ·  │
│     ·     │             ·  │
│ 249000241 │            175 │
│ 200057859 │            132 │
│ 249100546 │            192 │
│ 200056380 │             76 │
│ 200058485 │             85 │
│ 249710070 │            342 │
│ 249710047 │            160 │
│ 249720061 │            176 │
│ 249730045 │           4860 │
│ 249740119 │            289 │
├───────────┴────────────────┤
│    1255 rows (20 shown)    │
└────────────────────────────┘

In [24]:
query = """
SELECT 
    df_epci_filtered.*,
    df_surface_epci.superficie_km2
FROM df_epci_filtered
LEFT JOIN df_surface_epci
ON df_epci_filtered.siren = df_surface_epci.epci_code
"""

df_epci_filtered = duckdb.sql(query)

In [22]:
df_cat_nat = duckdb.read_csv("./data/raw/cat_nat.csv",skiprows=2)
df_cat_nat

┌───────────┬────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────┐
│   Code    │                          Libellé                           │ Nombre d'Arrêtés de Catastrophes Naturelles publiés au J.O. │
│   int64   │                          varchar                           │                            int64                            │
├───────────┼────────────────────────────────────────────────────────────┼─────────────────────────────────────────────────────────────┤
│ 200000172 │ CC Faucigny - Glières                                      │                                                          58 │
│ 200000438 │ CC du Pays de Pontchâteau St-Gildas-des-Bois               │                                                          45 │
│ 200000545 │ CC des Portes de Romilly sur Seine                         │                                                          31 │
│ 200000628 │ CC Rhône Lez Provence      

In [45]:
mapping = {
    "Code": "siren",
    "Libellé": "nom_epci",
    "Nombre d'Arrêtés de Catastrophes Naturelles publiés au J.O.": "nb_cat_nat"
}

df_cat_nat = df_cat_nat.df()  # conversion DuckDB → pandas
df_cat_nat = df_cat_nat.rename(columns=mapping)

df_cat_nat.loc[df_cat_nat['siren'] == 75056, 'siren'] = 200054781

df_cat_nat = duckdb.from_df(df_cat_nat)
df_cat_nat

┌───────────┬────────────────────────────────────────────────────────────┬────────────┐
│   siren   │                          nom_epci                          │ nb_cat_nat │
│   int64   │                          varchar                           │   int64    │
├───────────┼────────────────────────────────────────────────────────────┼────────────┤
│ 200000172 │ CC Faucigny - Glières                                      │         58 │
│ 200000438 │ CC du Pays de Pontchâteau St-Gildas-des-Bois               │         45 │
│ 200000545 │ CC des Portes de Romilly sur Seine                         │         31 │
│ 200000628 │ CC Rhône Lez Provence                                      │         87 │
│ 200000800 │ CC Coeur de Sologne                                        │         31 │
│ 200000925 │ CC de la Plaine Dijonnaise                                 │         91 │
│ 200000933 │ CC Sauldre et Sologne                                      │        111 │
│ 200004802 │ CC du Pays de Faye

In [51]:
query = """
SELECT e.siren,
        e.dept,
        e.total_pop_tot,
    df_cat_nat.nb_cat_nat,
    ROUND(1.0*df_cat_nat.nb_cat_nat / e.superficie_km2, 3) AS cat_nat_per_km2
FROM df_epci_filtered AS e
LEFT JOIN df_cat_nat
ON e.siren = df_cat_nat.siren
"""

df_cat_nat_final = duckdb.sql(query)

In [52]:
df_cat_nat_final.write_csv("./data/processed/cat_nat_epci.csv")